In [62]:
#Gear Design
import numpy as np
import pandas as pd
import math
from math import sqrt, pi

#General
T = 300 #F
Rel = 99
FOS = 2
Bearing_Life = 120 #kh

In [12]:
#Input 
Pinput = 150
Winput = 2000

#Cooling Pump
Ppump = 2
Wpump = 3000 #rpms

#Electrical Gen
Pgen = 53
Wgen = 1800

#Actuators/Motors
Pmotor = {'max':68, 'min':0}
Wmotor = 667

#Layout in
X1 = 12
X2 = 8
Yt = 12
Z1 = 4
Z2 = 8

i=1

#Test Ng
Ng = 60

In [93]:
eIC_motor = Winput/Wmotor
eIC_pump = Winput/Wpump
eIC_gen = Winput/Wgen

print("eIC+motor:", eIC_motor)
print("eIC+pump:",eIC_pump)
print("eIC+gen:",eIC_gen, "\n")

Np_eIC_pump = Ng/eIC_pump
Np_eIC_gen = Ng/eIC_gen
Np_eIC_motor = Ng/eIC_motor

print("Np(eIC+pump):",Np_eIC_pump)
print("Np(eIC+gen):",Np_eIC_gen)
print("Np(eIC+motor):",Np_eIC_motor, "\n")

#C = dp+dg/2
#e = dg/dp
#Ni = di*P

A = np.matrix([[0.5,0.5,0,0,0,0], [eIC_motor,1,0,0,0,0], [0,0,0.5,0.5,0,0],
        [0,0,-eIC_pump,1,0,0],[0,0,0,0,0.5,0.5],[0,0,0,0,-eIC_gen,1]])
# print(A, "\n")


B = np.matrix([[X1],[0],[sqrt(X2**2+((Yt/2)**2))],[0],[sqrt(X2**2+((Yt/2)**2))],[0],])
# print(B, "\n")

x = np.matmul(np.linalg.inv(A), B)

x = pd.DataFrame(x,index=(['dp_motor','dg_motor','dp_pump','dg_pump','dp_gen','dg_gen']))

Xmatrix = x[0]
print(Xmatrix, "\n")


P_pump = Np_eIC_pump/Xmatrix['dp_pump']
P_motor = Np_eIC_motor/Xmatrix['dp_motor']
P_gen = Np_eIC_gen/Xmatrix['dp_gen']

print("Pump:", P_pump)
print("Motor:",P_motor)
print("Gen:",P_gen)



eIC+motor: 2.998500749625187
eIC+pump: 0.6666666666666666
eIC+gen: 1.1111111111111112 

Np(eIC+pump): 90.0
Np(eIC+gen): 54.0
Np(eIC+motor): 20.01 

dp_motor   -12.009002
dg_motor    36.009002
dp_pump     12.000000
dg_pump      8.000000
dp_gen       9.473684
dg_gen      10.526316
Name: 0, dtype: float64 

Pump: 7.499999999999999
Motor: -1.6662499999999998
Gen: 5.7


In [81]:
dt = Xmatrix[0]